Add codes for EDA here and we'll convert them for Streamlit app later.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import geopandas as gpd
from shapely import wkt
import json


In [ ]:
from os import makedirs

makedirs("../jsonvis/",exist_ok=True)

In [ ]:
file_path = "../ProjectData/ChicagoCrimes(20152025).csv"
df = pd.read_csv(file_path)
df.head()

## Prepare Date and Time

Convert the 'Date' column to datetime objects with the specified format and `errors='coerce'` to handle potential parsing issues, and display the first few rows of the DataFrame to confirm the changes.




In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
df = df.rename(columns={"Date":"Datetime"})
df['Date'] = df['Datetime'].dt.date
df['Time'] = df['Datetime'].dt.time # pandas normalizes to 24-hour format
df.head()

## Understanding Dataset

In [ ]:
df.info()

In [ ]:
# Convert multiple columns at once to category type
categorical_columns = ['IUCR', 'Primary Type', 'Description',
                       'Location Description', 'Arrest', 'Domestic',
                       'Beat', 'District', 'Ward', 'Community Area']
df[categorical_columns] = df[categorical_columns].astype('category')

# Subset to only categorical columns and describe
df[categorical_columns].describe()

## Analyze Missing Data Over Time

Calculate the number and missing values for each column, grouped by year, to understand the trend of missing data over different years. This will provide insights into when certain data points started to be missing or were more prevalent.


In [ ]:
# Count number of cases per year
cases_per_year = df.groupby("Year").size()
print(cases_per_year)

In [ ]:
# Calculate missing values by year and column
missing_by_year = df.groupby("Year").apply(lambda x: x.isnull().sum())

# Plot heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(missing_by_year.T, cmap="Reds", annot=True, fmt="d", cbar_kws={'label': 'Fraction Missing'})
plt.title("Heatmap of Missing Values by Column and Year")
plt.xlabel("Year")
plt.ylabel("Columns")
plt.tight_layout()
plt.show()

## Analyse Duplicate Records

In [ ]:
# Check for duplicate Case Numbers
print("Duplicate Case Numbers Check:")
print("=" * 50)

# Count total duplicates
duplicate_count = df['Case Number'].duplicated().sum()
print(f"Total duplicate Case Numbers: {duplicate_count}")

# Check if any duplicates exist
has_duplicates = df['Case Number'].duplicated().any()
print(f"Has duplicates: {has_duplicates}")

# Count unique vs total
print(f"\nTotal rows: {len(df)}")
print(f"Unique Case Numbers: {df['Case Number'].nunique()}")
print(f"Duplicate rows: {len(df) - df['Case Number'].nunique()}")

# View duplicate Case Numbers
duplicate_case_numbers = df[df['Case Number'].duplicated(keep=False)]
print(f"\nRows with duplicate Case Numbers: {len(duplicate_case_numbers)}")

# Show duplicate Case Numbers with their counts
duplicate_summary = df['Case Number'].value_counts()
duplicate_summary = duplicate_summary[duplicate_summary > 1]
print(f"\nNumber of Case Numbers appearing more than once: {len(duplicate_summary)}")
print("\nTop 10 most duplicated Case Numbers:")
print(duplicate_summary.head(10))

# View actual duplicate rows
print("\nSample of duplicate rows:")
print(duplicate_case_numbers.sort_values('Case Number').head(10))

## Remove Missing Data, 2026 Data and Duplicates

Total enteries is 2755021. Fraction of missing data is small which will not distort EDA when removed. Also, this simplifies workflow - avoid complexity of imputing values which can introduce bias if not done carefully. Plots and summaries will not be cluttered by NaN values too.

Also removed 2026 data such that EDA will focus on years 2015 - 2025.

Also removed duplicate records, to retain only one entry per Case Number.


In [ ]:
print("Original entries:", len(df))
# Remove entries from the year 2026
df_clean = df[df["Year"] != 2026]
print("After dropping 2026 data:", len(df_clean))

# Drop rows with any missing values
df_clean = df_clean.dropna()
print("After dropping missing data:", len(df_clean))

# 'ID' is a separate column, sort first then drop duplicates
df_clean = df_clean.sort_values('ID', ascending=False)
df_clean = df_clean.drop_duplicates(subset='Case Number', keep='first')
print("After dropping duplicates based on Case Number:", len(df_clean))

## Time Series Plot by Date, Week, Month and Year

In [ ]:
# Ensure notebook renders Plotly charts
pio.renderers.default = "notebook"

# Assuming df_clean already has a 'Datetime' column parsed
df_clean["Date"] = df_clean["Datetime"].dt.date
df_clean["Week"] = df_clean["Datetime"].dt.to_period("W").dt.start_time
df_clean["Month"] = df_clean["Datetime"].dt.to_period("M").dt.start_time
df_clean["Year"] = df_clean["Datetime"].dt.year

# Aggregate counts using df_clean
cases_by_date = df_clean.groupby("Date").size().reset_index(name="Cases")
cases_by_week = df_clean.groupby("Week").size().reset_index(name="Cases")
cases_by_month = df_clean.groupby("Month").size().reset_index(name="Cases")
cases_by_year = df_clean.groupby("Year").size().reset_index(name="Cases")

# Build figure with multiple traces
fig = go.Figure()

fig.add_trace(go.Scatter(x=cases_by_date["Date"], y=cases_by_date["Cases"],
                         mode="lines", name="By Date", visible=True))
fig.add_trace(go.Scatter(x=cases_by_week["Week"], y=cases_by_week["Cases"],
                         mode="lines", name="By Week", visible=False))
fig.add_trace(go.Scatter(x=cases_by_month["Month"], y=cases_by_month["Cases"],
                         mode="lines", name="By Month", visible=False))
fig.add_trace(go.Scatter(x=cases_by_year["Year"], y=cases_by_year["Cases"],
                        mode="lines", name="By Year", visible=False))

# Add dropdown menu to toggle visibility
fig.update_layout(
    updatemenus=[
        dict(
            type="dropdown",
            x=0.97, y=1.12,
            buttons=[
                dict(label="Date", method="update",
                     args=[{"visible": [True, False, False, False]},
                           {"title": "Cases by Date"}]),
                dict(label="Week", method="update",
                     args=[{"visible": [False, True, False, False]},
                           {"title": "Cases by Week"}]),
                dict(label="Month", method="update",
                     args=[{"visible": [False, False, True, False]},
                           {"title": "Cases by Month"}]),
                dict(label="Year", method="update",
                     args=[{"visible": [False, False, False, True]},
                           {"title": "Cases by Year"}]),
            ]
        )
    ]
)

fig.show()


In [ ]:
fig.write_json("../jsonvis/time_series_seasonality.json")

### Observations

Outlier number of crime incidents reported on specific dates. High number of crime 1877 reported on 31 May 2020. Low number of crime 110 and 115 on 19 Dec 2023 and 14 May 2024 respectively.

Strictly lesser than 20k of crime incidents reported per month from Jan 2020 to May 2022. The lesser amount of crime might be due to Covid Pandemic effect.

Appears to be some seasonality trend or pattern for crime rate annually.

## Chicago Crime Heatmap by Time of Day and Weekday

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


df_clean['DayOfWeek'] = df_clean['Datetime'].dt.day_name()
df_clean['Hour'] = df_clean['Datetime'].dt.hour

DAY_ORDER = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
TOP_10_TYPES = df_clean['Primary Type'].value_counts().head(10).index


def prepare_heatmap_data(dataframe):
    heat = (
        dataframe
        .groupby(['DayOfWeek', 'Hour'])
        .size()
        .reset_index(name='Crime Count')
    )

    pivot = (
        heat
        .pivot(index='Hour', columns='DayOfWeek', values='Crime Count')
        .reindex(columns=DAY_ORDER)
        .fillna(0)
    )

    return pivot


def build_hourly_heatmap(pivot, title):
    return go.Heatmap(
        z=pivot.values,
        x=pivot.columns,
        y=pivot.index,
        colorscale='Reds',
        name=title,
        visible=False,
        colorbar=dict(
            title='Crime Count',
            thickness=14,
            len=0.85,
            y=0.5
        ),
        hovertemplate=(
            'Day: %{x}<br>'
            'Hour: %{y}:00<br>'
            'Crime Count: %{z}<extra></extra>'
        )
    )


fig = make_subplots(rows=1, cols=1)

# Overall
pivot_overall = prepare_heatmap_data(df_clean)
fig.add_trace(build_hourly_heatmap(pivot_overall, 'Overall'), row=1, col=1)

# Crime Types
for crime_type in TOP_10_TYPES:
    df_type = df_clean[df_clean['Primary Type'] == crime_type]
    pivot_type = prepare_heatmap_data(df_type)
    fig.add_trace(build_hourly_heatmap(pivot_type, crime_type), row=1, col=1)


buttons = [
    dict(
        label='Overall',
        method='update',
        args=[
            {'visible': [True] + [False]*len(TOP_10_TYPES)},
            {'title': 'Diurnal Crime Pattern by Day of Week (Overall)'}
        ]
    )
]

for i, crime_type in enumerate(TOP_10_TYPES):
    visibility = [False]*(1 + len(TOP_10_TYPES))
    visibility[1+i] = True

    buttons.append(
        dict(
            label=crime_type,
            method='update',
            args=[
                {'visible': visibility},
                {'title': f'Diurnal Crime Pattern — {crime_type}'}
            ]
        )
    )


fig.update_layout(
    updatemenus=[
        dict(
            type='dropdown',
            x=1,
            y=1.12,
            xanchor='right',
            buttons=buttons
        )
    ],
    height=620,
    margin=dict(l=60, r=60, t=80, b=60),
    font=dict(
        family='Arial',
        size=12
    ),
)

# Axis styling
fig.update_xaxes(
    title='Day of Week',
    categoryorder='array',
    categoryarray=DAY_ORDER,
    showgrid=False
)

fig.update_yaxes(
    title='Hour of Day',
    showgrid=False
)

fig.data[0].visible = True
fig.show()

In [ ]:
fig.write_json("../jsonvis/diurnal_heatmap.json")

## Crime Incidents Located Outside of Chicago

In [ ]:
# Validate that latitude is between -90 and 90 and longtitude is between -180 and 180
print(df_clean["Latitude"].min(), df_clean["Latitude"].max())
print(df_clean["Longitude"].min(), df_clean["Longitude"].max())


The min and max of Latitude and Longitude is beyond Chicago city boundaries. Present of outlier. Code below to identify/visualise outlier records.

In [ ]:
# Create the plot
plt.figure(figsize=(12, 8))

# Plot the data points
plt.scatter(df_clean['Longitude'], df_clean['Latitude'], 
            alpha=0.1, s=1, color='blue', label='Data Points')

# 5. Draw the "Real Chicago" Bounding Box for reference
# (Lat: 41.64 to 42.02 | Long: -87.94 to -87.52)
plt.axhline(y=41.64, color='red', linestyle='--', linewidth=1)
plt.axhline(y=42.02, color='red', linestyle='--', linewidth=1)
plt.axvline(x=-87.94, color='red', linestyle='--', linewidth=1)
plt.axvline(x=-87.52, color='red', linestyle='--', linewidth=1)

# 6. Formatting
plt.title('Identifying Lat/Long Outliers in Chicago Crime Data')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True, linestyle=':', alpha=0.6)
plt.legend(['Data Points', 'Actual Chicago Boundaries'], loc='upper left')

# Add text labels to help explain the chart
plt.text(-91, 38, "Outliers found here\n(Likely errors or state-wide data)", 
         color='darkred', fontsize=10, bbox=dict(facecolor='white', alpha=0.5))

plt.tight_layout()
# plt.savefig('outlier_detection_plot.png', dpi=300)
plt.show()

# 7. Print a summary of how many records are "Out of Bounds"
outliers = df_clean[
    (df_clean['Latitude'] < 41.6) | (df_clean['Latitude'] > 42.1) | 
    (df_clean['Longitude'] < -88.0) | (df_clean['Longitude'] > -87.5)
]

print(f"Total records: {len(df_clean)}")
print(f"Number of outlier records: {len(outliers)}")
print(f"Percentage of outliers: {(len(outliers)/len(df_clean))*100:.2f}%")

In [ ]:
# Filter out anything outside the Chicago city limits
df_map = df_clean[(df_clean['Latitude'] > 41.6) & (df_clean['Latitude'] < 42.1) & 
        (df_clean['Longitude'] > -88.0) & (df_clean['Longitude'] < -87.5)]
df_map.shape

# Chicago Choropleth Crime Density Map Over The Years

In [ ]:
df_polygon = pd.read_csv('../ProjectData/ChicagoCommunityArea.csv').iloc[:,1:]
df_polygon.columns = ['GEOMETRY','AREA_NUMBER','COMMUNITY','AREA_NUM_1','SHAPE_AREA','SHAPE_LEN']

In [ ]:
# Convert WKT string to geometry objects and convert area to km2
df_polygon["GEOMETRY_OBJ"] = df_polygon["GEOMETRY"].apply(wkt.loads)
df_polygon['SHAPE_AREA_FLT'] = df_polygon['SHAPE_AREA'].apply(lambda x: float(x.replace('.', '').replace(',', '.')))
df_polygon['AREA_KM2']  = round(df_polygon['SHAPE_AREA_FLT'] * 9.2903e-8,2)

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(
    df_polygon,
    geometry="GEOMETRY_OBJ",
    crs="EPSG:4326"
)

In [ ]:
df_crime_yearly = df_clean.groupby(['Community Area','Year']).size().reset_index(name='Total Crime')


gdf_plot_yearly = gdf.copy().merge(
    df_crime_yearly,
    left_on="AREA_NUMBER",
    right_on="Community Area",
    how="left"
)

gdf_plot_yearly['Crime/km2'] = round(gdf_plot_yearly['Total Crime'] / (gdf_plot_yearly['AREA_KM2']),0)

In [ ]:
geojson = json.loads(gdf.copy().to_json()) 
fig_choropleth_overall = px.choropleth_mapbox(
    gdf_plot_yearly, 
    geojson=geojson, 
    locations="AREA_NUMBER",
    featureidkey="properties.AREA_NUMBER", 
    color="Crime/km2", animation_frame="Year", 
    color_continuous_scale="Reds", 
    mapbox_style="open-street-map", 
    center={"lat": 41.828, "lon": -87.62}, 
    zoom=9, 
    range_color=(0, max(gdf_plot_yearly['Crime/km2'])), 
    opacity=0.85, 
    hover_name="COMMUNITY", 
    hover_data={"Total Crime": True, "AREA_KM2": True}, 
    height=650, 
    width=1000, 
    title="Crime Density by Community Area Over The Years" )

fig_choropleth_overall.show()


In [ ]:
fig_choropleth_overall.write_json("../jsonvis/crime_choropleth_map.json")

### Concentration of Crime in Top Community Areas - Heatmap

### Top Crime Type Heatmap for Top Community Areas (by Year)

In [ ]:
# Create a mapping dictionary from gdf
community_mapping = dict(zip(gdf['AREA_NUMBER'], gdf['COMMUNITY']))

# Add community name column to df_clean
df_clean['Community Name'] = df_clean['Community Area'].map(community_mapping)



In [ ]:
# Interactive Crime Type Heatmap by Year - Top 10 Community Areas
years = sorted(df_clean['Year'].unique())

# Get overall top 10 communities and top 10 crime types (consistent across years)
top_10_communities = df_clean.groupby('Community Name').size().nlargest(10).index
top_10_crimes = df_clean['Primary Type'].value_counts().head(10).index

# Find global max for consistent color scale
all_counts = []
for year in years:
    df_year = df_clean[df_clean['Year'] == year]
    year_counts = (
        df_year[
            (df_year['Community Name'].isin(top_10_communities)) & 
            (df_year['Primary Type'].isin(top_10_crimes))
        ]
        .groupby(['Community Name', 'Primary Type'])
        .size()
        .reset_index(name='Count')
    )
    all_counts.append(year_counts['Count'].max())

global_max = max(all_counts)

# Create initial heatmap for first year
df_year = df_clean[df_clean['Year'] == years[0]]
heatmap_data = (
    df_year[
        (df_year['Community Name'].isin(top_10_communities)) & 
        (df_year['Primary Type'].isin(top_10_crimes))
    ]
    .groupby(['Community Name', 'Primary Type'])
    .size()
    .reset_index(name='Count')
    .pivot(index='Community Name', columns='Primary Type', values='Count')
    .fillna(0)
)

heatmap_data = heatmap_data.reindex(
    index=top_10_communities,
    columns=top_10_crimes,
    fill_value=0
)

fig = go.Figure(data=[
    go.Heatmap(
        z=heatmap_data.values,
        x=heatmap_data.columns,
        y=heatmap_data.index,
        colorscale='YlOrRd',
        text=heatmap_data.values,
        texttemplate='%{text:.0f}',
        textfont={"size": 10},
        colorbar=dict(title='Count'),
        hovertemplate='<b>Community Area:</b> %{y}<br><b>Crime Type:</b> %{x}<br><b>Count:</b> %{z}<extra></extra>',
        zmin=0,
        zmax=global_max
    )
])

# Create frames for animation
frames = []
for year in years:
    df_year = df_clean[df_clean['Year'] == year]
    
    heatmap_data = (
        df_year[
            (df_year['Community Name'].isin(top_10_communities)) & 
            (df_year['Primary Type'].isin(top_10_crimes))
        ]
        .groupby(['Community Name', 'Primary Type'])
        .size()
        .reset_index(name='Count')
        .pivot(index='Community Name', columns='Primary Type', values='Count')
        .fillna(0)
    )
    
    heatmap_data = heatmap_data.reindex(
        index=top_10_communities,
        columns=top_10_crimes,
        fill_value=0
    )
    
    frame = go.Frame(
        data=[
            go.Heatmap(
                z=heatmap_data.values,
                x=heatmap_data.columns,
                y=heatmap_data.index,
                colorscale='YlOrRd',
                text=heatmap_data.values,
                texttemplate='%{text:.0f}',
                textfont={"size": 10},
                colorbar=dict(title='Count'),
                hovertemplate='<b>Community Area:</b> %{y}<br><b>Crime Type:</b> %{x}<br><b>Count:</b> %{z}<extra></extra>',
                zmin=0,
                zmax=global_max
            )
        ],
        name=str(year)
    )
    frames.append(frame)

fig.frames = frames

# Create slider steps with animate method
steps = []
for year in years:
    step = dict(
        method='animate',
        args=[
            [str(year)],
            {
                'frame': {'duration': 0, 'redraw': True},
                'mode': 'immediate',
                'transition': {'duration': 0, 'easing': 'linear'}
            }
        ],
        label=str(year)
    )
    steps.append(step)

# Add play/pause buttons
updatemenus = [dict(
    buttons=[
        dict(
            args=[None, {
                'frame': {'duration': 500, 'redraw': True},
                'mode': 'immediate',
                'fromcurrent': True,
                'transition': {'duration': 500, 'easing': 'linear'}
            }],
            label='▶',
            method='animate'
        ),
        dict(
            args=[[None], {
                'frame': {'duration': 0, 'redraw': True},
                'mode': 'immediate',
                'fromcurrent': True,
                'transition': {'duration': 0, 'easing': 'linear'}
            }],
            label='⏹',
            method='animate'
        )
    ],
    direction='right',
    pad=dict(r=10, t=10),
    showactive=False,
    type='buttons',
    x=0.01,
    xanchor='left',
    y=-0.6,
    yanchor='bottom'
)]

# Add slider control
sliders = [dict(
    active=0,
    currentvalue=dict(prefix='Year: '),
    len=0.85,
    pad=dict(b=10, t=10),
    x=0.2,
    xanchor='left',
    y=-0.7,
    yanchor='bottom',
    steps=steps
)]

fig.update_layout(
    updatemenus=updatemenus,
    sliders=sliders,
    title=f'Crime Type Heatmap - Top 10 Community Areas ({years[0]} - {years[-1]})',
    # xaxis_title='Crime Type',
    yaxis_title='Community Area',
    yaxis=dict(categoryorder='array', categoryarray=top_10_communities, autorange='reversed'),
    height=650,
    width=1100,
    margin=dict(t=100, b=150, l=200, r=100)
)

fig.update_xaxes(tickangle=45)
fig.show()

In [ ]:
fig.write_json("../jsonvis/area_crimetype_heatmap.json")

## Top 10 Crime Type Distribution Over The Years

In [ ]:
# Group by Primary Type and Year (without Month)
df_crime_type_annual = df_clean.groupby(['Primary Type', 'Year']).size().reset_index(name='Crime Count')

# Get top 10 crime types by total count across all years
top_crimes = df_crime_type_annual.groupby('Primary Type')['Crime Count'].sum().nlargest(10).index
df_top10_annual = df_crime_type_annual[df_crime_type_annual['Primary Type'].isin(top_crimes)]

# Create animated bar chart
fig_top_crime_annual = px.bar(
    df_top10_annual,
    x="Crime Count",
    y="Primary Type",
    animation_frame="Year",
    orientation="h",
    title="Top 10 Crime Types (by Year)",
    color="Primary Type"
)

fig_top_crime_annual.update_layout(
    xaxis_title="Crime Count (Annual)",
    yaxis_title="Crime Type",
    height=800,
    width=1000,
    showlegend=True,
    xaxis=dict(range=[0, df_top10_annual['Crime Count'].max() * 1.1])  # Add 10% padding
)

fig_top_crime_annual.update_xaxes(tickformat=",d")

# Ensure bars are sorted descending in every frame
fig_top_crime_annual.update_yaxes(categoryorder="total ascending")

fig_top_crime_annual.show()

In [ ]:
fig_top_crime_annual.write_json("../jsonvis/top_crime_annual.json")

## Arrest Rate by Crime Type

In [ ]:
from matplotlib.patches import Patch

In [ ]:
# Ensure Arrest is numeric (0/1) before aggregation
df_clean['Arrest_Num'] = df_clean['Arrest'].astype(int)

arrest_rate = (
    df_clean.groupby('Primary Type')['Arrest_Num']
    .mean()
    .mul(100)
    .reset_index(name='Arrest Rate (%)')
    .sort_values('Arrest Rate (%)', ascending=False)
)

# Create color palette: red for top crimes, blue for others
top_crimes_list = list(top_crimes)
colors = [
    'red' if crime in top_crimes_list else 'steelblue'
    for crime in arrest_rate['Primary Type']
]


# Plot arrest rates (highest at top)
plt.figure(figsize=(10, 6))
sns.barplot(
    data=arrest_rate,
    x='Arrest Rate (%)',
    y='Primary Type',
    order=arrest_rate['Primary Type'],
    # palette='viridis'
    palette=colors
)

legend_elements = [
    Patch(facecolor='red', label='Top 10 Crimes'),
    Patch(facecolor='steelblue', label='Other Crimes')
]

plt.legend(handles=legend_elements, loc='lower right')
plt.title('Arrest Rate by Crime Type')
plt.xlabel('Arrest Rate (%)')
plt.ylabel('Crime Type')
plt.tight_layout()
plt.savefig("arrest_rate.png", dpi=300, bbox_inches="tight")
plt.show()